In [21]:
import pandas as pd
import numpy as np


import requests
import urllib.request
import time
from bs4 import BeautifulSoup

import re

import spotipy

import spotipy.util as util

from fuzzywuzzy import fuzz

In [22]:
# enter website info

url = 'https://www.bandsintown.com'

response = requests.get(url)

In [23]:
# check to see if we have soup...  ('200' means that we have a valid connection)

response

<Response [200]>

In [24]:
# start the soup

soup = BeautifulSoup(response.text, 'html.parser')

In [25]:
# make soup outta that website!

soup.findAll('a')

[<a class="_1c0VJLyfwWzjJfOHu6eP_U" href="https://www.bandsintown.com/?came_from=257"><img alt="Bandsintown Fist" src="https://assets.bandsintown.com/images/bitFist.svg"/><img alt="Bandsintown Text" class="_1EY9FymTCu4aZ1GqRcBdh2" src="https://assets.bandsintown.com/images/bitText.svg"/></a>,
 <a class="_1ig2tgu-gwv3YNa5JaLJyj"><div class="_1L7wm0rQie9wLuUaXd38C7"><img alt="Phone icon" src="https://assets.bandsintown.com/images/blackPhone.svg"/></div><div class="_1EpztBXJ4yePIdqCEbohk1">Get App</div></a>,
 <a href="https://manager.bandsintown.com">Artists &amp; Venues</a>,
 <a href="https://promoters.bandsintown.com">Promoters</a>,
 <a href="https://help.bandsintown.com">Help</a>,
 <a href="https://corp.bandsintown.com/privacy">Privacy</a>,
 <a class="_1-03HGg4F65cN9_ILhg4yX" href="https://corp.bandsintown.com/terms">Terms</a>,
 <a href="https://help.bandsintown.com/">Help Center</a>,
 <a class="_3DfSNPsAeJTWxgocTU6DpT" href="https://www.bandsintown.com/?came_from=257&amp;livestream=tr

In [26]:
# find and inspect first elemnt from 'event's near' info from bottoom of website

event_urls = soup.findAll('a', '_3UX9sLQPbNUbfbaigy35li')

event_urls[0]


<a class="_3UX9sLQPbNUbfbaigy35li" href="https://www.bandsintown.com/e/102119169-black-pool-at-soma?came_from=257&amp;utm_medium=web&amp;utm_source=home&amp;utm_campaign=event"></a>

In [27]:
# grab the 'event's near' info from bottom of website

pattern = re.compile(r"\b\S*-at\b")

reg_list = re.findall(pattern , str(event_urls))

print(reg_list)

['href="https://www.bandsintown.com/e/102119169-black-pool-at', 'href="https://www.bandsintown.com/e/102025858-little-stranger-at', 'href="https://www.bandsintown.com/e/102097644-christopher-hollyday-at', 'href="https://www.bandsintown.com/e/102082581-seun-anikulapo-kuti-at', 'href="https://www.bandsintown.com/e/102133449-adelitas-way-at', 'href="https://www.bandsintown.com/e/102158189-him-+-her-worship-at', 'href="https://www.bandsintown.com/e/102221009-payton-howie-at', 'href="https://www.bandsintown.com/e/102169772-colleen-green-at', 'href="https://www.bandsintown.com/e/102180458-shane-gillis-at', 'href="https://www.bandsintown.com/e/101919337-chad-lefkowitz-brown-at', 'href="https://www.bandsintown.com/e/1018630814-robert-cray-at', 'href="https://www.bandsintown.com/e/101906159-big-jay-oakerson-at', 'href="https://www.bandsintown.com/e/102016987-bonneville-7-at', 'href="https://www.bandsintown.com/e/102025576-rdgldgrn-at', 'href="https://www.bandsintown.com/e/102137260-sheila-sonde

In [28]:
# select the link from the original grab

for i, b in enumerate(reg_list) :

    reg_list[i] = b.replace('-at','').replace('-', ' ')
    
reg_list

['href="https://www.bandsintown.com/e/102119169 black pool',
 'href="https://www.bandsintown.com/e/102025858 little stranger',
 'href="https://www.bandsintown.com/e/102097644 christopher hollyday',
 'href="https://www.bandsintown.com/e/102082581 seun anikulapo kuti',
 'href="https://www.bandsintown.com/e/102133449 adelitas way',
 'href="https://www.bandsintown.com/e/102158189 him + her worship',
 'href="https://www.bandsintown.com/e/102221009 payton howie',
 'href="https://www.bandsintown.com/e/102169772 colleen green',
 'href="https://www.bandsintown.com/e/102180458 shane gillis',
 'href="https://www.bandsintown.com/e/101919337 chad lefkowitz brown',
 'href="https://www.bandsintown.com/e/1018630814 robert cray',
 'href="https://www.bandsintown.com/e/101906159 big jay oakerson',
 'href="https://www.bandsintown.com/e/102016987 bonneville 7',
 'href="https://www.bandsintown.com/e/102025576 rdgldgrn',
 'href="https://www.bandsintown.com/e/102137260 sheila sondergard (sheson)',
 'href="htt

In [29]:
# select the band name after the space 
# this should work even for bands that start with numbers!

pattern = re.compile(r"\s(.*)")

for i, b in enumerate(reg_list):
    
    reg_list[i] = re.findall(pattern, str(b))

reg_list    

[['black pool'],
 ['little stranger'],
 ['christopher hollyday'],
 ['seun anikulapo kuti'],
 ['adelitas way'],
 ['him + her worship'],
 ['payton howie'],
 ['colleen green'],
 ['shane gillis'],
 ['chad lefkowitz brown'],
 ['robert cray'],
 ['big jay oakerson'],
 ['bonneville 7'],
 ['rdgldgrn'],
 ['sheila sondergard (sheson)'],
 ['speaker honey'],
 ['tim apple'],
 ['r.o. shapiro']]

In [30]:
# get rid of parentheses and commas (and nested lists by default)

for i, b in enumerate(reg_list) :

    reg_list[i] = b[0].replace("'",'').replace(',', '')
    
reg_list

['black pool',
 'little stranger',
 'christopher hollyday',
 'seun anikulapo kuti',
 'adelitas way',
 'him + her worship',
 'payton howie',
 'colleen green',
 'shane gillis',
 'chad lefkowitz brown',
 'robert cray',
 'big jay oakerson',
 'bonneville 7',
 'rdgldgrn',
 'sheila sondergard (sheson)',
 'speaker honey',
 'tim apple',
 'r.o. shapiro']

In [31]:
# drop duplicates by using a set

band_list = list(set(reg_list))

band_list

['payton howie',
 'christopher hollyday',
 'tim apple',
 'r.o. shapiro',
 'robert cray',
 'little stranger',
 'seun anikulapo kuti',
 'him + her worship',
 'big jay oakerson',
 'rdgldgrn',
 'chad lefkowitz brown',
 'sheila sondergard (sheson)',
 'adelitas way',
 'colleen green',
 'bonneville 7',
 'black pool',
 'speaker honey',
 'shane gillis']

In [32]:
client_id = ''
    
client_secret = ''

username = ''

scope = ''

redirect_uri = ''

In [33]:
token = util.prompt_for_user_token(username, scope,
                                   client_id=client_id,
                                   client_secret=client_secret,
                                   redirect_uri=redirect_uri) 

sp = spotipy.Spotify(auth=token)

In [34]:
playlist_name = 'Upcoming Local Shows Playlist'   

description = 'Here are some hot songs from artists that will be playing in the area soon!'

playlist = sp.user_playlist_create(username, name=playlist_name, description=description )


In [35]:
def grab_track_ids(band_list):
    
    
    track_ids = []
    
    
    for i in range(len(band_list)):
        
        results = sp.search(q=f"{band_list[i]}", limit=3, type='track') 
        
        if results['tracks']['total'] == 0: 
            
            continue
        
        else:
            
            for j in range(len(results['tracks']['items'])):
                
                if fuzz.partial_ratio(results['tracks']['items'][j]['artists'][0]['name'], band_list[i]) > 60: 
                    
                    track_ids.append(results['tracks']['items'][j]['id']) 

                
                else:
                    continue
  
    print("Gottem!")
    
    return track_ids

In [36]:
track_ids = grab_track_ids(band_list)

track_ids

Gottem!


['4JndhJccdHJUYPjdMUTeY1',
 '1H3TIcL6kCOwqR8wjqs6Ii',
 '7kZ0rM9Lp0Ib7rPF7XebNM',
 '34zN49oCGAEvMiI521JNm8',
 '7FrB7HSdPIpLp5CCCk1GzV',
 '0PNl2I8v2yK0dWbk1kb1WY',
 '4hprs9pNIAnazIOHyoimon',
 '5tRu8kA4d17uwGVT7I3YIR',
 '23YyyUiEZYlT3MkzpZhRbW',
 '51R3YhOIXXaCPcvdB9k21j',
 '7rKSR3gY70SH8AM8fswBge',
 '79ZN5n4pQfYf3DPGoOD0Tq',
 '3ZDzVMZoFTx1gMbQszGy6d',
 '2ZVvoSR6j0uzAOHEYhp9VR',
 '74nLzZ6fimLWkBvzX1JuEV',
 '3P1Y9RIYsMk87Kdmt7hQtO',
 '2WCNV7XBKv5P58Y7tprWF0',
 '2DeBp0xumahsCVdRfBgQvQ',
 '5kg4zCCt0oRJpZT2TR8DUG',
 '2Scp56JqNUByjPLq34AyGw',
 '7pP64UgtXGch0blNm5dIwB',
 '4xilOU0GV1EadnoFQKp20R',
 '5PZo9NMyNLxOlpEFoYo611',
 '4FthwGFz9SVZgCVqxNXsSK',
 '5f7CQQlWscLYxkdzaKWx5Q',
 '6iL0iOBkAUtKuCAo2ynYyh',
 '5pSd6GLg6xIPTGLitMrF6n',
 '6c1T4m4RcD1RMF7ROmcbh2',
 '4f1MOhX1eEls7hVsEEPKIO',
 '5SrfclddwYE4JkwY8UqF7H',
 '448etSVdv1hrsZyVnd4d3S',
 '1klF0nbl8zZPK822jNmLTB']

In [37]:
playlist_id = playlist['id']

playlist_id

'0xEIJXZz1MCFtvBepSxS8f'

In [38]:
sp.user_playlist_add_tracks(username, playlist_id, track_ids)

{'snapshot_id': 'Myw3ZGVhODAwYTQ3YjJmOWQyNzcxMGViNTNiZDA5OWU3ODg1YTQ5M2Qy'}